# Limitations of Unsupervised Methods

In the previous two sections, we saw examples of the K-Means and Principal Component Analysis (PCA) methods, both of which are examples of unsupervised learning.